# DiskANN 索引详解教程

本教程将深入讲解 Milvus 中的 **DiskANN** 索引，包括其原理、创建流程、参数配置和检索优化。

## 什么是 DiskANN？

**DiskANN** (Disk-based Approximate Nearest Neighbor) 是微软研究院开发的一种基于磁盘的近似最近邻搜索算法。

### 核心优势

| 特性 | 说明 |
|------|------|
| 🔥 **低内存占用** | 将索引数据存储在磁盘上，大幅减少内存需求 |
| ⚡ **高性能** | 通过图结构优化，实现毫秒级检索 |
| 📈 **高召回率** | 在保证速度的同时，维持较高的搜索精度 |
| 💾 **大规模支持** | 适合十亿级别的向量数据集 |

### 适用场景

- 数据量大、内存有限的场景
- 需要在成本和性能之间取得平衡
- 推荐系统、图像检索、文档搜索等应用

## 环境要求

- Milvus 2.0+ (Docker 单机版或集群版)
- pymilvus >= 2.3.0
- 足够的磁盘空间（用于存储索引）

## 1. 环境准备与连接

首先导入必要的库并连接到 Milvus 服务器。

> ⚠️ **注意**: DiskANN 索引需要 Milvus 单机版 (Docker) 或集群版，不支持 Milvus Lite。

In [19]:
import numpy as np
from pymilvus import (
    MilvusClient,
    DataType,
)

# 连接到 Milvus Docker 单机版
# 确保 Docker 中的 Milvus 服务已启动
MILVUS_URI = "http://localhost:19530"

client = MilvusClient(uri=MILVUS_URI)

print(f"✅ 成功连接到 Milvus!")
print(f"📦 服务器版本: {client.get_server_version()}")
print(f"📋 当前集合列表: {client.list_collections()}")

✅ 成功连接到 Milvus!
📦 服务器版本: 2.6.8
📋 当前集合列表: ['diskann_tutorial', 'diskann_demo']


## 2. 创建集合 (Collection)

创建一个用于存储向量数据的集合。我们需要定义 Schema，包括：
- **主键字段**: 唯一标识每条数据
- **向量字段**: 存储向量嵌入
- **标量字段**: 存储元数据（可选）

### Schema 定义说明

```python
schema.add_field(
    field_name="vector",           # 字段名称
    datatype=DataType.FLOAT_VECTOR,# 数据类型：浮点向量
    dim=DIMENSION                  # 向量维度
)
```

In [20]:
# 配置参数
COLLECTION_NAME = "diskann_tutorial"
DIMENSION = 128  # 向量维度

# 如果集合已存在，先删除（便于重复测试）
if client.has_collection(COLLECTION_NAME):
    client.drop_collection(COLLECTION_NAME)
    print(f"⚠️ 已删除旧集合: {COLLECTION_NAME}")

# 创建 Schema
schema = client.create_schema(
    auto_id=False,           # 不自动生成 ID，手动指定
    enable_dynamic_field=True # 允许动态字段（灵活添加元数据）
)

# 添加主键字段
schema.add_field(
    field_name="id",
    datatype=DataType.INT64,
    is_primary=True,
    description="主键 ID"
)

# 添加向量字段
schema.add_field(
    field_name="vector",
    datatype=DataType.FLOAT_VECTOR,
    dim=DIMENSION,
    description="向量嵌入"
)

# 添加标量字段（用于过滤）
schema.add_field(
    field_name="category",
    datatype=DataType.VARCHAR,
    max_length=64,
    description="分类标签"
)

# 创建集合
client.create_collection(
    collection_name=COLLECTION_NAME,
    schema=schema,
    consistency_level="Strong"  # 强一致性
)

print(f"✅ 成功创建集合: {COLLECTION_NAME}")
print(f"📊 集合列表: {client.list_collections()}")

⚠️ 已删除旧集合: diskann_tutorial
✅ 成功创建集合: diskann_tutorial
📊 集合列表: ['diskann_tutorial', 'diskann_demo']


### 查看集合详情

使用 `describe_collection` 查看集合的完整配置信息。

In [21]:
# 查看集合详情
collection_info = client.describe_collection(COLLECTION_NAME)

print("📋 集合配置信息:")
print(f"  - 集合名称: {collection_info['collection_name']}")
print(f"  - 自动生成ID: {collection_info['auto_id']}")
print(f"  - 动态字段: {collection_info['enable_dynamic_field']}")
print(f"  - 字段数量: {len(collection_info['fields'])}")

print("\n📊 字段详情:")
for field in collection_info['fields']:
    print(f"  - {field['name']}: {field['type']}")

📋 集合配置信息:
  - 集合名称: diskann_tutorial
  - 自动生成ID: False
  - 动态字段: True
  - 字段数量: 3

📊 字段详情:
  - id: 5
  - vector: 101
  - category: 21


## 3. 插入向量数据

在创建索引之前，需要先插入数据。我们生成一些模拟的向量数据用于测试。

### 数据格式说明

Milvus 支持以下数据插入格式：
- **字典列表**: `[{"id": 1, "vector": [...], "category": "A"}, ...]`
- **列式数据**: `{"id": [1, 2, ...], "vector": [[...], [...]], "category": ["A", "B", ...]}`

In [22]:
# 生成测试数据
NUM_ENTITIES = 10000  # 数据量
CATEGORIES = ["electronics", "clothing", "books", "food", "sports"]

np.random.seed(42)  # 固定随机种子，确保可复现

# 生成随机向量
vectors = np.random.random((NUM_ENTITIES, DIMENSION)).astype(np.float32)

# 准备插入数据
data = [
    {
        "id": i,
        "vector": vectors[i].tolist(),
        "category": CATEGORIES[i % len(CATEGORIES)],  # 循环分配类别
        "price": round(np.random.uniform(10, 1000), 2),  # 动态字段：价格
        "in_stock": bool(np.random.randint(0, 2))  # 动态字段：是否有货
    }
    for i in range(NUM_ENTITIES)
]

print(f"📦 准备插入 {NUM_ENTITIES} 条数据")
print(f"📊 数据示例:")
print(f"  - ID: {data[0]['id']}")
print(f"  - 向量维度: {len(data[0]['vector'])}")
print(f"  - 类别: {data[0]['category']}")
print(f"  - 价格: {data[0]['price']}")
print(f"  - 有货: {data[0]['in_stock']}")

📦 准备插入 10000 条数据
📊 数据示例:
  - ID: 0
  - 向量维度: 128
  - 类别: electronics
  - 价格: 189.9
  - 有货: True


In [23]:
# 批量插入数据
# 对于大数据量，建议分批插入（每批 1000-10000 条）
BATCH_SIZE = 5000

for i in range(0, NUM_ENTITIES, BATCH_SIZE):
    batch = data[i:i + BATCH_SIZE]
    res = client.insert(
        collection_name=COLLECTION_NAME,
        data=batch
    )
    print(f"✅ 批次 {i // BATCH_SIZE + 1}: 插入 {len(batch)} 条数据")

# 刷新数据到磁盘（确保统计信息正确）
client.flush(collection_name=COLLECTION_NAME)
print("💾 数据已刷新到磁盘")

# 获取集合统计
stats = client.get_collection_stats(COLLECTION_NAME)
print(f"\n📊 集合总数据量: {stats['row_count']} 条")

✅ 批次 1: 插入 5000 条数据
✅ 批次 2: 插入 5000 条数据
💾 数据已刷新到磁盘

📊 集合总数据量: 10000 条


## 4. 创建 DiskANN 索引 ⭐

这是本教程的**核心部分**。DiskANN 索引的创建需要在插入数据之后进行。

### DiskANN 索引参数详解

| 参数 | 说明 | 建议值 |
|------|------|--------|
| `index_type` | 索引类型，必须为 `"DISKANN"` | `"DISKANN"` |
| `metric_type` | 距离度量方式 | `"L2"` 或 `"IP"` |

### 距离度量方式

- **L2 (欧氏距离)**: 适合大多数场景，值越小越相似
- **IP (内积)**: 适合归一化向量，值越大越相似
- **COSINE (余弦相似度)**: 适合文本向量，值越大越相似

> 💡 **提示**: DiskANN 目前支持 L2 和 IP 距离度量。

In [24]:
# 准备索引参数
index_params = client.prepare_index_params()

# 添加 DiskANN 索引
index_params.add_index(
    field_name="vector",      # 向量字段名称
    index_type="DISKANN",     # 索引类型: DiskANN
    metric_type="L2",         # 距离度量: L2 欧氏距离
    params={}                 # DiskANN 使用默认参数
)

print("📋 索引参数配置:")
print(f"  - 字段: vector")
print(f"  - 索引类型: DISKANN")
print(f"  - 度量方式: L2 (欧氏距离)")

📋 索引参数配置:
  - 字段: vector
  - 索引类型: DISKANN
  - 度量方式: L2 (欧氏距离)


In [25]:
import time

# 创建索引
print("🔧 开始创建 DiskANN 索引...")
start_time = time.time()

client.create_index(
    collection_name=COLLECTION_NAME,
    index_params=index_params
)

elapsed = time.time() - start_time
print(f"✅ DiskANN 索引创建完成! 耗时: {elapsed:.2f} 秒")

🔧 开始创建 DiskANN 索引...
✅ DiskANN 索引创建完成! 耗时: 2.13 秒


### 查看索引信息

创建索引后，可以查看索引的详细信息，包括构建状态和参数。

In [26]:
# 查看索引信息
index_info = client.describe_index(
    collection_name=COLLECTION_NAME,
    index_name="vector"  # 索引名称默认与字段名相同
)

print("📊 DiskANN 索引信息:")
print(f"  - 索引名称: {index_info.get('index_name', 'vector')}")
print(f"  - 索引类型: {index_info['index_type']}")
print(f"  - 度量类型: {index_info['metric_type']}")
print(f"  - 字段名称: {index_info['field_name']}")
print(f"  - 构建状态: {index_info.get('state', 'Unknown')}")
print(f"  - 总行数: {index_info.get('total_rows', 'N/A')}")
print(f"  - 已索引行数: {index_info.get('indexed_rows', 'N/A')}")

📊 DiskANN 索引信息:
  - 索引名称: vector
  - 索引类型: DISKANN
  - 度量类型: L2
  - 字段名称: vector
  - 构建状态: Finished
  - 总行数: 10000
  - 已索引行数: 10000


## 5. 加载集合

在进行搜索之前，必须先将集合加载到内存中。

> 💡 **DiskANN 特点**: 虽然索引数据存储在磁盘上，但仍需要将部分元数据加载到内存中以支持高效搜索。

In [27]:
# 加载集合到内存
print("📥 加载集合到内存...")
client.load_collection(collection_name=COLLECTION_NAME)
print(f"✅ 集合 {COLLECTION_NAME} 已加载，可以开始搜索！")

📥 加载集合到内存...
✅ 集合 diskann_tutorial 已加载，可以开始搜索！


## 6. 向量检索 (Search) ⭐

现在可以使用 DiskANN 索引进行向量检索了。

### DiskANN 搜索参数详解

| 参数 | 说明 | 取值范围 | 建议 |
|------|------|----------|------|
| `search_list` | 搜索列表大小 | 建议 ≥ `limit` | 100-200 |

**search_list 参数说明**:
- 值越大，搜索越精确，但速度越慢
- 值越小，搜索越快，但可能降低召回率
- 推荐值：`limit` 的 2-10 倍

In [28]:
# 生成查询向量
query_vector = np.random.random((1, DIMENSION)).astype(np.float32).tolist()

# DiskANN 搜索参数
search_params = {
    "metric_type": "L2",      # 必须与索引的 metric_type 一致
    "params": {
        "search_list": 100    # DiskANN 特有参数，控制搜索精度
    }
}

# 执行向量搜索
print("🔍 执行向量搜索...")
start_time = time.time()

results = client.search(
    collection_name=COLLECTION_NAME,
    data=query_vector,         # 查询向量
    limit=5,                   # 返回 Top 5
    search_params=search_params,
    output_fields=["id", "category", "price", "in_stock"]  # 返回的字段
)

elapsed = time.time() - start_time
print(f"✅ 搜索完成! 耗时: {elapsed*1000:.2f} ms")

# 显示搜索结果
print("\n📊 搜索结果 (Top 5):")
print("-" * 60)
for i, hit in enumerate(results[0]):
    print(f"  排名 {i+1}:")
    print(f"    - ID: {hit['id']}")
    print(f"    - 距离: {hit['distance']:.4f}")
    print(f"    - 类别: {hit['entity']['category']}")
    print(f"    - 价格: ${hit['entity']['price']}")
    print(f"    - 有货: {'是' if hit['entity']['in_stock'] else '否'}")

🔍 执行向量搜索...
✅ 搜索完成! 耗时: 12.29 ms

📊 搜索结果 (Top 5):
------------------------------------------------------------
  排名 1:
    - ID: 5697
    - 距离: 13.7674
    - 类别: books
    - 价格: $304.78
    - 有货: 否
  排名 2:
    - ID: 2095
    - 距离: 14.1327
    - 类别: electronics
    - 价格: $783.97
    - 有货: 否
  排名 3:
    - ID: 292
    - 距离: 14.1444
    - 类别: books
    - 价格: $639.88
    - 有货: 否
  排名 4:
    - ID: 3483
    - 距离: 14.1570
    - 类别: food
    - 价格: $707.74
    - 有货: 是
  排名 5:
    - ID: 7228
    - 距离: 14.1843
    - 类别: food
    - 价格: $403.22
    - 有货: 是


### 验证搜索准确性

使用一个已知向量进行搜索，验证是否能正确找到自己（距离应为 0）。

In [29]:
# 使用 ID=0 的已知向量进行搜索
known_vector = vectors[0].tolist()

results = client.search(
    collection_name=COLLECTION_NAME,
    data=[known_vector],
    limit=5,
    search_params=search_params,
    output_fields=["id", "category"]
)

print("🎯 使用 ID=0 的向量搜索验证:")
print("-" * 40)
for hit in results[0]:
    is_self = "✓ 找到自己!" if hit['id'] == 0 else ""
    print(f"  ID: {hit['id']}, 距离: {hit['distance']:.6f} {is_self}")

🎯 使用 ID=0 的向量搜索验证:
----------------------------------------
  ID: 0, 距离: 0.000000 ✓ 找到自己!
  ID: 9162, 距离: 13.837808 
  ID: 1589, 距离: 14.800013 
  ID: 7333, 距离: 15.083131 
  ID: 8110, 距离: 15.309496 


## 5. 高级检索：带过滤条件的搜索

DiskANN 支持在向量搜索的同时应用标量过滤条件，这是混合检索的重要能力：

### 过滤表达式语法

| 操作符 | 说明 | 示例 |
|--------|------|------|
| `==` | 等于 | `category == "A"` |
| `!=` | 不等于 | `category != "A"` |
| `in` | 包含于 | `category in ["A", "B"]` |
| `>`, `<`, `>=`, `<=` | 比较运算 | `id > 100` |
| `and`, `or` | 逻辑运算 | `id > 100 and category == "A"` |

In [30]:
# 带过滤条件的搜索：只在 category == "electronics" 的数据中搜索
filtered_results = client.search(
    collection_name=COLLECTION_NAME,
    data=query_vector,  # query_vector 已经是 [[...]] 格式，不需要再加 []
    limit=5,
    search_params=search_params,
    filter='category == "electronics"',  # 过滤条件
    output_fields=["id", "category"]
)

print("🔍 带过滤条件的搜索 (category == 'electronics'):")
print("-" * 50)
for hit in filtered_results[0]:
    print(f"  ID: {hit['id']}, Category: {hit['entity']['category']}, 距离: {hit['distance']:.4f}")

# 验证所有结果都属于 electronics 类别
all_electronics = all(hit['entity']['category'] == "electronics" for hit in filtered_results[0])
print(f"\n✓ 所有结果都属于 electronics 类别: {all_electronics}")

🔍 带过滤条件的搜索 (category == 'electronics'):
--------------------------------------------------
  ID: 2095, Category: electronics, 距离: 14.1327
  ID: 2225, Category: electronics, 距离: 14.3197
  ID: 6715, Category: electronics, 距离: 14.4439
  ID: 3290, Category: electronics, 距离: 14.4922
  ID: 4495, Category: electronics, 距离: 14.5413

✓ 所有结果都属于 electronics 类别: True


## 6. search_list 参数对比实验

`search_list` 是 DiskANN 最重要的性能调优参数。让我们通过实验来理解它的影响：

**理论分析：**
- `search_list` 越大 → 搜索更多候选节点 → 召回率更高 → 但速度更慢
- `search_list` 越小 → 搜索更少候选节点 → 速度更快 → 但可能遗漏相似结果

**建议值范围：** 通常在 topK ~ 200 之间，根据精度需求调整

In [31]:
import time

# 测试不同 search_list 值对搜索的影响
search_list_values = [10, 20, 50, 100, 200]

print("📊 search_list 参数对比实验:")
print("=" * 60)
print(f"{'search_list':<15} {'搜索时间(ms)':<15} {'结果数':<10}")
print("-" * 60)

for sl in search_list_values:
    params = {"metric_type": "L2", "params": {"search_list": sl}}
    
    # 多次搜索取平均时间
    times = []
    for _ in range(10):
        start = time.time()
        results = client.search(
            collection_name=COLLECTION_NAME,
            data=query_vector,  # query_vector 已经是 [[...]] 格式
            limit=10,
            search_params=params
        )
        times.append((time.time() - start) * 1000)
    
    avg_time = sum(times) / len(times)
    print(f"{sl:<15} {avg_time:<15.3f} {len(results[0]):<10}")

print("-" * 60)
print("💡 结论: search_list 增大会略微增加搜索时间，但可提高召回率")

📊 search_list 参数对比实验:
search_list     搜索时间(ms)        结果数       
------------------------------------------------------------
10              1.849           10        
20              2.294           10        
50              2.678           10        
100             4.085           10        
200             7.324           10        
------------------------------------------------------------
💡 结论: search_list 增大会略微增加搜索时间，但可提高召回率


## 7. 资源清理与最佳实践

在生产环境中，合理管理资源非常重要：

### 资源管理 API

| 操作 | 方法 | 说明 |
|------|------|------|
| 释放集合 | `release_collection()` | 从内存中卸载，但保留磁盘数据 |
| 删除集合 | `drop_collection()` | 完全删除集合及其所有数据 |
| 关闭连接 | `close()` | 关闭客户端连接 |

### DiskANN 使用最佳实践

1. **数据量要求：** DiskANN 适合大规模数据（>100万向量），小数据集可考虑 IVF_FLAT 或 HNSW
2. **内存规划：** DiskANN 索引在磁盘上，但查询时需要部分内存作为缓存
3. **search_list 调优：** 从较小值开始（如20），根据召回率需求逐步增加
4. **批量操作：** 尽量使用批量插入和批量搜索，减少网络开销

In [32]:
# 清理资源
print("🧹 清理资源...")

# 1. 释放集合（从内存卸载）
client.release_collection(COLLECTION_NAME)
print(f"  ✓ 集合 '{COLLECTION_NAME}' 已从内存释放")

# 2. 删除集合（可选，取消注释以执行）
client.drop_collection(COLLECTION_NAME)
print(f"  ✓ 集合 '{COLLECTION_NAME}' 已删除")

# 3. 关闭连接
client.close()
print("  ✓ 客户端连接已关闭")

print("\n🎉 DiskANN 教程完成！")

🧹 清理资源...
  ✓ 集合 'diskann_tutorial' 已从内存释放
  ✓ 集合 'diskann_tutorial' 已删除
  ✓ 客户端连接已关闭

🎉 DiskANN 教程完成！


## 📝 总结

本教程完整介绍了 Milvus DiskANN 索引的使用流程：

### 核心流程回顾

```
1. 连接 Milvus 服务器
       ↓
2. 创建集合（定义 Schema）
       ↓
3. 插入向量数据
       ↓
4. 创建 DiskANN 索引
       ↓
5. 加载集合到内存
       ↓
6. 执行向量搜索
       ↓
7. 清理资源
```

### DiskANN 关键参数

| 参数 | 阶段 | 作用 |
|------|------|------|
| `metric_type` | 索引创建 | 距离度量方式（L2/IP/COSINE） |
| `search_list` | 搜索 | 控制搜索精度与速度的权衡 |
| `limit` | 搜索 | 返回的最近邻数量 |
| `filter` | 搜索 | 标量过滤条件 |

### 适用场景

- ✅ 超大规模数据集（>1亿向量）
- ✅ 内存受限的环境
- ✅ 对召回率要求高的场景
- ❌ 小规模数据集（建议用 HNSW）
- ❌ 需要极低延迟的场景

---
*更多信息请参考 [Milvus 官方文档](https://milvus.io/docs)*